### Разработка нейросетевых функций. Операция Convolution 2D

#### Описание параметров

in_channels (int) – Number of channels in the input image\
out_channels (int) – Number of channels produced by the convolution\
kernel_size (int or tuple) – Size of the convolving kernel\
stride (int or tuple, optional) – Stride of the convolution. Default: 1\
padding (int, tuple or str, optional) – Padding added to all four sides of the input. Default: 0\
padding_mode (str, optional) – 'zeros', 'reflect', 'replicate' or 'circular'. Default: 'zeros'\
dilation (int or tuple, optional) – Spacing between kernel elements. Default: 1\
groups (int, optional) – Number of blocked connections from input channels to output channels. Default: 1\
bias (bool, optional) – If True, adds a learnable bias to the output. Default: True\

In [1]:
import torch
import numpy as np
def myConv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, 
             dilation=1, groups=1, bias=True, padding_mode='zeros'):
    def Conv(matrix):
        if (in_channels%groups != 0) or (out_channels%groups!=0):
            raise Exception('in_channel и out_channel не делятся на группы!')
    
        if bias == True:
            bias_val = torch.rand(out_channels)
        else:
            bias_val = torch.zeros(out_channels)

        if (padding_mode == 'zeros'):
            pad = torch.nn.ZeroPad2d(padding)
            matrix = pad(matrix)
        if (padding_mode == 'reflect'):
            pad = torch.nn.ReflectionPad2d(padding)
            matrix = pad(matrix)
        if (padding_mode == 'replicate'):
            pad = torch.nn.ReplicationPad2d(2)
            matrix = pad(matrix)
        if (padding_mode == 'circular'):
            pad = torch.nn.CircularPad2d(padding)
            matrix = pad(matrix)

        if type(kernel_size) == tuple:
            filter = torch.rand(out_channels, in_channels//groups, kernel_size[0], kernel_size[1])
        if type(kernel_size) == int:
            filter = torch.rand(out_channels, in_channels//groups, kernel_size, kernel_size)
            

        li = []
        for l in range(out_channels):
            feature_map = np.array([])
            for i in range (0, matrix.shape[1]-((filter.shape[2]-1)*dilation+1)+1, stride):
                for j in range (0, matrix.shape[2]-((filter.shape[3]-1)*dilation+1)+1, stride):
                    summ = 0
                    for c in range (in_channels//groups): #groups
                        if groups>1:
                            val = matrix[l*(in_channels//groups)+c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+
                                                                    (filter.shape[3]-1)*dilation+1:dilation]
                        else:
                            val = matrix[c][i:i+(filter.shape[2]-1)*dilation+1:dilation, j:j+(filter.shape[3]-1)*
                                            dilation+1:dilation]
                        mini_sum = (val*filter[l][c]).sum()
                        summ = summ + mini_sum
                    feature_map = np.append(feature_map, float(summ + bias_val[l]))

            li.append(feature_map.reshape((matrix.shape[1]-((filter.shape[2]-1)*dilation+1))//stride+1, 
                                          (matrix.shape[2]-((filter.shape[3]-1)*dilation+1))//stride+1))

        return np.array(li), np.array(filter), np.array(bias_val)

    return Conv

### Тестирование функции

In [5]:
img1 = torch.rand(4, 5, 6)
img2 = torch.rand(3, 28, 28)
img3 = torch.rand(1, 1, 1)

Func = myConv2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2,
                bias=True, padding_mode='zeros')
result, kernel, bias_val = Func(img1)
torchFunction = torch.nn.Conv2d(in_channels=4, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2,
                                bias=True, padding_mode='zeros')
torchFunction.weight.data = torch.tensor(kernel)
torchFunction.bias.data = torch.tensor(bias_val)
myResult = str(np.round(result, 2))
torchResult = str(np.round(np.array(torchFunction(img1).data), 2))
torchResult == myResult

True

In [6]:
Func2 = myConv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1, 
                 bias=True, padding_mode='zeros')
result2, kernel2, bias_val2 = Func2(img2)
torchFunction2 = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=3, stride=1, padding=0, dilation=2, groups=1,
                                 bias=True, padding_mode='zeros')
torchFunction2.weight.data = torch.tensor(kernel2)
torchFunction2.bias.data = torch.tensor(bias_val2)
myResult = str(np.round(result2, 2))
torchResult = str(np.round(np.array(torchFunction2(img2).data), 2))
torchResult == myResult

True

In [7]:
Func3 = myConv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, 
                 bias=True, padding_mode='zeros')
result3, kernel3, bias_val3 = Func3(img3)
torchFunction3 = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1,
                                 bias=True, padding_mode='zeros')
torchFunction3.weight.data = torch.tensor(kernel3)
torchFunction3.bias.data = torch.tensor(bias_val3)
myResult = str(np.round(result3, 2))
torchResult = str(np.round(np.array(torchFunction3(img3).data), 2))
torchResult == myResult

True